[🥭 Mango Markets](https://mango.markets/) support is available at: [Docs](https://docs.mango.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/mangomarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/blockworks-foundation/mango-explorer-examples/HEAD?labpath=ShowPerpFillsAndOrders.ipynb) [Run this code](https://mybinder.org/v2/gh/blockworks-foundation/mango-explorer-examples/HEAD?labpath=ShowPerpFillsAndOrders.ipynb) on Binder.

_🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

# 🥭 Show Perp Fills And Orders

This notebook shows how to fetch the fills and orders for a given account on a given market.

It loads the data from Solana in a single call so that all fetched data is in sync.

In [ ]:
import mango

# Use our hard-coded devnet wallet for DeekipCw5jz7UgQbtUbHQckTYGKXWaPQV4xY93DaiM6h.
# For real-world use you'd load the bytes from the environment or a file.
wallet = mango.Wallet(bytes([67,218,68,118,140,171,228,222,8,29,48,61,255,114,49,226,239,89,151,110,29,136,149,118,97,189,163,8,23,88,246,35,187,241,107,226,47,155,40,162,3,222,98,203,176,230,34,49,45,8,253,77,136,241,34,4,80,227,234,174,103,11,124,146]))

# Create a 'devnet' Context
with mango.ContextBuilder.build(cluster_name="devnet") as context:
    # Mango accounts are per-Group, so we need to load the Group first.
    group = mango.Group.load(context)

    # Get all the Wallet's accounts for that Group
    accounts = mango.Account.load_all_for_owner(context, wallet.address, group)

    # Just pick the first account even if there are multiple accounts
    account = accounts[0]

    # Load the market
    market = mango.market(context, "BTC-PERP")

    # Gather all addresses for the fetch
    bids_address = market.bids_address
    asks_address = market.asks_address
    event_queue_address = market.event_queue_address

    fetched = mango.AccountInfo.load_multiple(context, [bids_address, asks_address, event_queue_address])

    orderbook = market.parse_account_infos_to_orderbook(fetched[0], fetched[1])
    event_queue = mango.PerpEventQueue.parse(fetched[2], market.lot_size_converter)

    orders = list([o for o in [*orderbook.bids, *orderbook.asks] if o.owner == account.address])

    print("Orders:", orders)
    print("Fills:", event_queue.fills_for_account(account.address))

print("Example complete.")